<a href="https://colab.research.google.com/github/jstangelbellarmine/GaiaEye/blob/main/FotoMapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connects to drive and uploads a picture from my drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/SkokieB.JPG'
print("Using file:", file_path)

Mounted at /content/drive
Using file: /content/drive/MyDrive/SkokieB.JPG


Allows exifread, which will be used later to extract gps information.

In [ ]:
!pip install exifread

import exifread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.4 MB/s eta 0:00:00


This section reads the data to see if there is any information on GPS coordinates. Hey, look at that, who would have known that a picture I took in 2022 would help me a project 3 years later!

In [ ]:
with open(file_path, 'rb') as f:
    tags = exifread.process_file(f)

print("Available EXIF tags:")
for tag in tags:
    print(f"{tag}: {tags[tag]}")

required_tags = ['GPS GPSLatitude', 'GPS GPSLatitudeRef', 'GPS GPSLongitude', 'GPS GPSLongitudeRef']

if all(tag in tags for tag in required_tags):
    print("GPS data is available in the image.")
else:
    print("GPS data is not available in the image.")


Available EXIF tags:
Image Make: Apple
Image Model: iPhone 13 Pro
Image Orientation: Rotated 90 CW
Image XResolution: 72
Image YResolution: 72
Image ResolutionUnit: Pixels/Inch
Image Software: 15.6.1
Image DateTime: 2022:09:16 13:07:51
Image HostComputer: iPhone 13 Pro
Image YCbCrPositioning: Centered
Image ExifOffset: 236
GPS GPSLatitudeRef: N
GPS GPSLatitude: [42, 6, 1757/100]
GPS GPSLongitudeRef: W
GPS GPSLongitude: [87, 44, 5731/100]
GPS GPSAltitudeRef: 0
GPS GPSAltitude: 168135/884
GPS GPSSpeedRef: K
GPS GPSSpeed: 0
GPS GPSImgDirectionRef: T
GPS GPSImgDirection: 624809/1767
GPS GPSDestBearingRef: T
GPS GPSDestBearing: 624809/1767
GPS GPSDate: 2022:09:16
GPS Tag 0x001F: 426345/49144
Image GPSInfo: 2474
Thumbnail Compression: JPEG (old-style)
Thumbnail XResolution: 72
Thumbnail YResolution: 72
Thumbnail ResolutionUnit: Pixels/Inch
Thumbnail JPEGInterchangeFormat: 2842
Thumbnail JPEGInterchangeFormatLength: 15830
EXIF ExposureTime: 1/121
EXIF FNumber: 3/2
EXIF ExposureProgram: Progra


This section creates easy-to-read numbers for the kml file

In [ ]:
def get_decimal_from_dms(dms, ref):
    """
    Convert GPS coordinates in degrees, minutes, seconds to decimal degrees.
    """
    degrees = dms[0]
    minutes = dms[1]
    seconds = dms[2]

    decimal = float(degrees.num) / float(degrees.den) + \
              float(minutes.num) / (float(minutes.den) * 60) + \
              float(seconds.num) / (float(seconds.den) * 3600)

    if ref in ['S', 'W']:
        decimal = -decimal
    return decimal

This section extracts the GPS coordinates if they exist

In [ ]:
with open(file_path, 'rb') as f:
    tags = exifread.process_file(f)

try:
    gps_latitude = tags['GPS GPSLatitude']
    gps_latitude_ref = tags['GPS GPSLatitudeRef'].printable
    gps_longitude = tags['GPS GPSLongitude']
    gps_longitude_ref = tags['GPS GPSLongitudeRef'].printable

    lat = get_decimal_from_dms(gps_latitude.values, gps_latitude_ref)
    lon = get_decimal_from_dms(gps_longitude.values, gps_longitude_ref)

    print("Latitude:", lat)
    print("Longitude:", lon)
except KeyError:
    print("No GPS data found in this image.")

Latitude: 42.10488055555556
Longitude: -87.74925277777778


This section creates the KML file once the coordinates have been extracted.

In [ ]:
try:
    kml_content = f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Placemark>
    <name>Photo Location</name>
    <Point>
      <coordinates>{lon},{lat},0</coordinates>
    </Point>
  </Placemark>
</kml>
'''

    kml_filename = "photo_location.kml"
    with open(kml_filename, "w") as file:
        file.write(kml_content)

    print("KML file created:", kml_filename)
except NameError:
    print("KML file not created due to missing GPS data.")

KML file created: photo_location.kml


Saves the file

In [ ]:
from google.colab import files
files.download(kml_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>